<a href="https://colab.research.google.com/github/elainedias16/AI/blob/main/Roteiro_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ******* LIBRARIES **********

# Basic libraries for data manipulation
import pandas as pd
from bs4 import BeautifulSoup

# Libraries used for reading database
from posixpath import defpath
import requests
import os
import chardet

# Libraries used for preprocessing
import nltk
import re
import unicodedata
nltk.download('stopwords')
from nltk.corpus import stopwords
## Stemming
from nltk.stem import RSLPStemmer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('rslp')
## Lemmanization
import spacy
!python -m spacy download pt_core_news_sm

In [ ]:
def remove_tags(html):
    # parse html content
    soup = BeautifulSoup(html, "html.parser")

    for data in soup(['style', 'script']):
        # Remove tags
        data.decompose()

    # return data by retrieving the tag content
    return ' '.join(soup.stripped_strings)


# Inicializar o stemmer RSLPStemmer
# RSLP is a pt stemmer, Porter is one of the most used en stemmers
# http://text-processing.com/demo/stem/
stemmer = RSLPStemmer()
def stemming(text):
  words = word_tokenize(text)
  stemmed_words = [stemmer.stem(word) for word in words]
  stemmed_sentence = ' '.join(stemmed_words)
  return stemmed_sentence


# nlp = spacy.load('en_core_web_sm')
nlp = spacy.load('pt_core_news_sm')
def lemmanization(text):
  doc = nlp(text)
  lemmatized_tokens = [token.lemma_ for token in doc]
  lemmatized_text = ' '.join(lemmatized_tokens)
  return lemmatized_text


In [ ]:
stop_words = set(stopwords.words('portuguese'))

def pre_processing(text):
  #Put space in non alphanumeric characteres
  text_without_non_alnum = ''
  for character in text:
    if character.isalnum():
      text_without_non_alnum += character
    else:
      text_without_non_alnum += ' '

  # Remove accents:
  text_without_accents = ''.join(c for c in unicodedata.normalize('NFD', text_without_non_alnum) if unicodedata.category(c) != 'Mn')

  # Remove duplicated spaces:
  text_without_duplicates = ' '.join(text_without_accents.split())

  # Lowercasting
  text_without_duplicates = text_without_duplicates.lower()

  # Remove stop words:
  words = nltk.word_tokenize(text_without_duplicates)
  text_without_stop_words = ''

  for word in words:
    if word not in stop_words:
      text_without_stop_words += word + ' ' #It's necessary to put space because tokenize takes away it

  # Lemmanization
  lemmatized_text = lemmanization(text_without_stop_words)

  # Stemming:
  stemmed_text = stemming(lemmatized_text)

  return stemmed_text


In [ ]:
id_path = '1TJTEKu5hxk25q36O6xF1OruLjySlW6_n'
directory_path = '/content/drive/MyDrive/dados-aTribuna-classes-selecionadas'

def create_dataframe():
  df = pd.DataFrame(columns=['genre', 'content'])
  folders = os.listdir(directory_path)

  for folder in folders:
    genre_path = directory_path + '/' + folder

    if os.path.isdir(genre_path): #There is a file in the root
        os.chdir(genre_path)
        for filename in os.listdir(genre_path):
            file_path = os.path.join(genre_path, filename)

            if filename.endswith('.txt'):
              with open(file_path, 'rb') as file:
                content = file.read()
                content_without_html_tags = remove_tags(content)
                content_pre_procesed = pre_processing(content_without_html_tags)
                # df = df.append({'genre': folder, 'content': content_pre_procesed}, ignore_index=True) #Append method will be deprecied soon
                df_to_concat = pd.DataFrame({'genre': folder, 'content': content_pre_procesed}, index=[0])
                df = pd.concat([df, df_to_concat], ignore_index=True)


  return df


df = create_dataframe()

df.head()
# Dividir em funções, parte que pega o nome dos arquivos e outra que faz o pré-processamento
# #colocar o nome do arquivo no dataframe
# mostrar o passo em um texto e ir printando

#BBC News Dataset

# Leitura do Dataset e crição do Dataframe

In [ ]:
import pandas as pd

id_file = '1m41gWGwnLYo0nAEOABnU_MH_SCQHWfjS'
url_file = 'https://drive.google.com/uc?id={}'.format(id_file)

def create_dataframe(path):
    columns=['genre', 'filename', 'title', 'content']
    df =  pd.read_table(
    path,
    header=None,
    names=columns,
    sep= '\t',
    engine= 'python'
    )
    return df


df = create_dataframe(url_file)
df = df.drop(0).reset_index(drop=True)

df.tail()

,genre,filename,title,content
2220,tech,397.txt,BT program to beat dialler scams,BT is introducing two initiatives to help bea...
2221,tech,398.txt,Spam e-mails tempt net shoppers,Computer users across the world continue to i...
2222,tech,399.txt,Be careful how you code,A new European directive could put software w...
2223,tech,400.txt,US cyber security chief resigns,The man making sure US computer networks are ...
2224,tech,401.txt,Losing yourself in online gaming,"Online role playing games are time-consuming,..."


In [ ]:
import nltk
nltk.download('punkt')
from nltk import sent_tokenize, word_tokenize

nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))


import spacy
from spacy.lang.en.examples import sentences

## Lemmanization
import spacy
# !python -m spacy download pt_core_news_sm
!pip install --upgrade spacy
# !python -m spacy download en_core_news_sm
!python -m spacy download en_core_web_sm


In [86]:
# !pip install unidecode
import unidecode
import spacy
from nltk.stem.porter import PorterStemmer


original = """
"The sun was shining brightly as Sarah walked through the park. Birds chirped in the trees, and a gentle breeze rustled the leaves. It was a perfect day for a leisurely stroll"
"""

def remove_non_alphanumeric(text):
    words = ''
    for w in text:
        words += w if w.isalnum() else ' '
    return words

def remove_accents(text):
  return unidecode.unidecode(text)


def lowercasting(tokens):
  tokens = [w.lower() for w in tokens]
  return tokens

def remove_stop_words(tokens):
  stop_words = set(stopwords.words('english')) #List of stop words in english
  tokens = [token for token in tokens if token not in stop_words]
  return tokens


def stemming(tokens):
  porter = PorterStemmer()
  tokens = [porter.stem(token) for token in tokens]
  return tokens


# Load the English language model
nlp = spacy.load("en_core_web_sm")

def lemmanization(text):
    doc = nlp(text)   # Process the text using the Spacy NLP
    words = [token.lemma_ for token in doc]
    return words


def pre_processing(text):
  tokens = remove_non_alphanumeric(text)
  tokens = remove_accents(tokens)
  tokens = word_tokenize(tokens)
  tokens = lowercasting(tokens)
  tokens = remove_stop_words(tokens)
  words = ' '.join(tokens)
  tokens = lemmanization(words)
  tokens = stemming(tokens)
  return ' '.join(tokens)

sentence = pre_processing(original)
print(sentence)



sun shine brightli sarah walk park bird chirp tree gentl breez rustl leav perfect day leisur stroll
